# googleの画像検索から画像をScrappingするための基本的なコード

In [1]:
import bs4
import requests
import re
import urllib.request, urllib.error
import os
import argparse
import sys
import json

#### 欲しい画像の種類と数と画像を保存するフォルダーを指定

In [2]:
search=['dog']
num_images=10
directry_name='./images'

#### 複数のキーワードを'+'でつなげる

In [3]:
if len(search) > 2:
    query = '+'.join(search)
else:
    query=search[0]

#### スクレイピングしてきた画像群を置けるように指定したフォルダー下にカテゴリー別に保存できるようにフォルダーをカテゴリー名で作成する

In [12]:
save_directory=[]
for folder_name in search:
    save_directory=directry_name+'/'+folder_name+'/'
    if not os.path.exists(save_directory):
        os.makedirs(save_directory) 

# スクレーピング

#### Google で画像検索した時にでるurl

In [5]:
url="https://www.google.co.jp/search?q="+query+"&source=lnms&tbm=isch"


#### 自分のUser-Agentを確認してそれをheaderに入力


In [6]:
header={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}

#### HTMLやXMLファイルからデータを取得するライブラリでsoupオブジェクトを取得
##### => soupにはHTMLやXMLの色々なデータが入っている。

In [7]:
soup=bs4.BeautifulSoup(urllib.request.urlopen(urllib.request.Request(url,headers=header)),'html.parser')

#### Actuallimagesに取得した画像のリンクとその画像の型を入れる。

In [10]:
ActualImages=[]
div_rg_meta=soup.find_all("div",class_="rg_meta")

for a in div_rg_meta:
    #a.text タグを取り除く　json.loads => JSON文字列を辞書に変換 
    #link　画像のリンク　Type　画像の型
    link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
    ActualImages.append((link,Type))

#### urlリンクから画像を取得し画像を指定のディレクトリにファイルを保存する。

In [15]:
for i , (img_link , Type) in enumerate( ActualImages[0:ｎum_images]):
    try:
        Type = Type if len(Type) > 0 else 'jpg'
        print("Downloading image {} ({}), type is {}".format(i, img_link, Type))
        #画像のリンクから、画像を取得する。
        raw_img = urllib.request.urlopen(img_link).read()
        #取得した画像を開いて保存する。
        f = open(os.path.join(save_directory , search[0]+"_"+str(i)+"."+Type), 'wb')
        f.write(raw_img)
        f.close()
    #loadができなかった場合エラーを吐くようにする。
    except Exception as e:
        print ("could not load : "+img)
        print (e)